In [4]:
import numpy as np
import scipy
from statsmodels.stats.proportion import proportion_confint

In [8]:
alpha = 0.003
z = scipy.stats.norm.ppf(1 - alpha / 2.)

print(z.round(4))

2.9677


In [16]:
sample1 = np.zeros((11037, ))
sample1[:104] = 1

sample2 = np.zeros((11034, ))
sample2[:189] = 1

p1 = sum(sample1) / len(sample1)
p2 = sum(sample2) / len(sample2)

print((p2 - p1).round(4))

0.0077


In [17]:
def proportions_confint_diff_ind(sample1, sample2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)   
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

In [20]:
left_boundary, right_boundary = proportions_confint_diff_ind(sample2, sample1, alpha = 0.05)

print(right_boundary.round(4))

0.0107


In [23]:
odds_plc = p2 / (1 - p2)
odds_asp = p1 / (1 - p1)

print((odds_plc / odds_asp).round(4))

1.8321


In [24]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

In [32]:
np.random.seed(0)

asp_bootstrap_samples = get_bootstrap_samples(sample1, 1000)
plc_bootstrap_samples = get_bootstrap_samples(sample2, 1000)

In [36]:
odds_asp_bootstrap = [ (sum(sample) / len(sample)) / (1 - (sum(sample) / len(sample))) for sample in asp_bootstrap_samples]

In [37]:
odds_plc_bootstrap = [ (sum(sample) / len(sample)) / (1 - (sum(sample) / len(sample))) for sample in plc_bootstrap_samples]

In [33]:
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [40]:
odds_scores = list(map(lambda x: x[1] / x[0], zip(odds_asp_bootstrap, odds_plc_bootstrap)))

In [41]:
boundaries = stat_intervals(odds_scores, 0.05)

In [43]:
print(boundaries[0].round(4))

1.4442
